In [1]:
# import NumPy for numerical operations
import numpy as np

# import pandas for data manipulation
import pandas as pd

In [2]:
# import SimpleImputer to handle missing values
from sklearn.impute import SimpleImputer

# import OrdinalEncoder to convert ordered categorical features into numerical format
from sklearn.preprocessing import OrdinalEncoder

# import OneHotEncoder for encoding categorical features into binary format
from sklearn.preprocessing import OneHotEncoder

In [3]:
# load the dataset from 'covid_toy.csv' into a DataFrame
df = pd.read_csv('covid_toy.csv')

In [4]:
# display the first 5 rows of the DataFrame
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
# display summary info: column names, data types, and non-null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


In [6]:
# count missing (null) values in each column
df.isnull().sum()

,0
age,0
gender,0
fever,10
cough,0
city,0
has_covid,0


## **Fever has null values --- SimpleImputer**

In [7]:
# count the number of occurrences for each category in the 'gender' column
df['gender'].value_counts()

,count
gender,
Female,59
Male,41


## **Gender is Nominal Categorical variable --- OneHotEncoding**

In [8]:
# count the number of occurrences for each category in the 'city' column
df['city'].value_counts()

,count
city,
Kolkata,32
Bangalore,30
Delhi,22
Mumbai,16


## **City is Nominal Categorical variable --- OneHotEncoding**

In [9]:
# count the number of occurrences for each category in the 'cough' column
df['cough'].value_counts()

,count
cough,
Mild,62
Strong,38


## **Cough is Ordinal Categorical variable --- OrdinalEncoding**

In [10]:
# count the number of occurrences for each category in the 'has_covid' column
df['has_covid'].value_counts()

,count
has_covid,
No,55
Yes,45


In [11]:
# import function to split data into training and testing sets
from sklearn.model_selection import train_test_split

# split the dataset into features and target
# features: all columns except 'has_covid'
# target: 'has_covid' column
# 80% training data, 20% testing data
x_train, x_test, y_train, y_test = train_test_split(
    df.drop(columns=['has_covid']),
    df['has_covid'],
    test_size=0.2
)

In [12]:
# display the training feature data
x_train

,age,gender,fever,cough,city
44,20,Male,102.0,Strong,Delhi
55,81,Female,101.0,Mild,Mumbai
79,48,Female,103.0,Mild,Kolkata
4,65,Female,101.0,Mild,Mumbai
68,54,Female,104.0,Strong,Kolkata
...,...,...,...,...,...
96,51,Female,101.0,Strong,Kolkata
97,20,Female,101.0,Mild,Bangalore
60,24,Female,102.0,Strong,Bangalore
24,13,Female,100.0,Strong,Kolkata


👉 Use fit_transform() on training data:
- It learns the transformation (e.g., encoding, imputation) using training data
- Then applies that transformation to the same data

👉 Use transform() on test (or new) data:
- It applies the same transformation learned from the training data
- Prevents data leakage and ensures consistency between training and testing

In [13]:
# create a SimpleImputer to fill missing values (default strategy is 'mean')
si = SimpleImputer()

# fit and transform 'fever' column in training data
x_train_fever = si.fit_transform(x_train[['fever']])

# transform 'fever' column in test data using the same imputer
x_test_fever = si.transform(x_test[['fever']])

# check the shape of the imputed 'fever' training data
x_train_fever.shape

(80, 1)

In [14]:
# display the imputed 'fever' values from training data
x_train_fever

array([[102.        ],
       [101.        ],
       [103.        ],
       [101.        ],
       [104.        ],
       [103.        ],
       [100.        ],
       [101.02777778],
       [104.        ],
       [104.        ],
       [102.        ],
       [102.        ],
       [ 98.        ],
       [ 99.        ],
       [100.        ],
       [104.        ],
       [100.        ],
       [ 98.        ],
       [101.02777778],
       [104.        ],
       [100.        ],
       [104.        ],
       [ 98.        ],
       [100.        ],
       [100.        ],
       [104.        ],
       [101.        ],
       [ 99.        ],
       [101.        ],
       [101.02777778],
       [103.        ],
       [ 98.        ],
       [101.        ],
       [104.        ],
       [101.02777778],
       [100.        ],
       [100.        ],
       [101.        ],
       [ 98.        ],
       [102.        ],
       [ 99.        ],
       [ 98.        ],
       [102.        ],
       [101

In [15]:
# create an OrdinalEncoder for the 'cough' column with custom order
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])

# fit and transform 'cough' column in training data
x_train_cough = oe.fit_transform(x_train[['cough']])

# transform 'cough' column in test data using the same encoder
x_test_cough = oe.transform(x_test[['cough']])

# check the shape of the encoded 'cough' training data
x_train_cough.shape

(80, 1)

In [16]:
# create OneHotEncoder to encode 'gender' and 'city', dropping the first category to avoid multicollinearity
# set sparse=False to get a dense NumPy array instead of a sparse matrix
ohe = OneHotEncoder(drop='first', sparse_output=False)

# fit and transform training data (learn encoding and apply it)
x_train_gender_city = ohe.fit_transform(x_train[['gender', 'city']])

# transform test data using the same encoding learned from training
x_test_gender_city = ohe.transform(x_test[['gender', 'city']])

# check the shape of the encoded training data
x_train_gender_city.shape

(80, 4)

In [17]:
# extract numerical columns (excluding 'gender', 'fever', 'cough', 'city') from training data
x_train_age = x_train.drop(columns=['gender', 'fever', 'cough', 'city']).values

# do the same for test data
x_test_age = x_test.drop(columns=['gender', 'fever', 'cough', 'city']).values

# check the shape of the extracted numeric training data
x_train_age.shape

(80, 1)

In [18]:
# combine all processed training features: age, fever, gender & city (one-hot), and cough (ordinal)
x_train_transformed = np.concatenate((x_train_age, x_train_fever, x_train_gender_city, x_train_cough), axis=1)

# combine all processed test features in the same order
x_test_transformed = np.concatenate((x_test_age, x_test_fever, x_test_gender_city, x_test_cough), axis=1)

# check the shape of the final transformed training feature matrix
x_train_transformed.shape

(80, 7)

## **Column Transformer**

In [19]:
# import ColumnTransformer to apply different preprocessing steps to different columns
from sklearn.compose import ColumnTransformer

In [20]:
# create a ColumnTransformer to apply different preprocessing to different columns
transformer = ColumnTransformer(transformers=[
    # impute missing values in 'fever' column using SimpleImputer
    ('tnf1', SimpleImputer(), ['fever']),

    # encode 'cough' column using OrdinalEncoder with custom category order
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),

    # one-hot encode 'gender' and 'city' columns, dropping the first category
    ('tnf3', OneHotEncoder(drop='first', sparse_output=False), ['gender', 'city'])

], remainder='passthrough')  # keep remaining columns (like 'age') as they are

In [21]:
# display the structure and configuration of the ColumnTransformer
transformer

ColumnTransformer(remainder='passthrough',
                  transformers=[('tnf1', SimpleImputer(), ['fever']),
                                ('tnf2',
                                 OrdinalEncoder(categories=[['Mild',
                                                             'Strong']]),
                                 ['cough']),
                                ('tnf3',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['gender', 'city'])])

In [22]:
# fit and transform the training data using the defined ColumnTransformer
transformer.fit_transform(x_train)

array([[102.        ,   1.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  20.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  81.        ],
       [103.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  48.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  65.        ],
       [104.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  54.        ],
       [103.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  50.        ],
       [100.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  19.        ],
       [101.02777778,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  20.        ],
       [104.        ,   1.        ,   0.        ,   1.        ,
          0.    

In [23]:
# apply the same transformations to the test data (without refitting)
transformer.transform(x_test)

array([[ 98.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,   5.        ],
       [103.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  60.        ],
       [101.02777778,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  84.        ],
       [102.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  82.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  25.        ],
       [ 99.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  14.        ],
       [ 99.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  72.        ],
       [103.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  73.        ],
       [101.02777778,   1.        ,   0.        ,   0.        ,
          0.    

In [24]:
# check the shape of the transformed training data
transformer.transform(x_train).shape

(80, 7)

In [25]:
# check the shape of the transformed test data
transformer.transform(x_test).shape

(20, 7)